# 1. Introduction

### 1.1. Starting Point

In machine learning, our goal is to find the weights (parameters) of a model (e.g., linear regression) that minimize the error between our predictions and the actual values. This involves optimizing the weights to make the model's predictions as accurate as possible.

For linear regression, we commonly use the squared error as a measure of how far off our predictions are. The objective is to adjust the weights to minimize this error. For a single data point, the difference between the predicted value and the actual value is referred to as the **loss**: 

$$
(f(x_{i}) - y_{i})^2
$$

When calculating the error across all data points, this becomes the **cost function**:

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (f(x_{i}) - y_{i})^2
$$

where $ m $ is the number of data points, and we divide by $ 2m $ instead of just $ m $ to simplify later calculations.


### 1.2. Minimizing the Cost Function

The cost function depends on the weights of the model, typically denoted as $ \theta$ (Theta) in machine learning. Our objective is to minimize the cost function with respect to $ \theta $.

To minimize any function, the first step is to compute its derivative. In this case, because we have multiple variables (weights), we compute the partial derivative with respect to each weight while treating the other weights as constants.


### 1.3. Gradient Descent: Deriving the Update Rules

Let’s start with the cost function for linear regression:

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (\theta_{0} + \theta_{1}X_{1i} - y_{i})^2
$$

Where:
- $ \theta_0 $ is the intercept,
- $ \theta_1 $ is the weight associated with predictor $ X_1 $.

Expanding the squared term:

$$
(\theta_{0} + \theta_{1}X_{1i} - y_{i})^2 = \theta_{0}^2 + 2\theta_{0}\theta_{1}X_{1i} + \theta_{1}^2X_{1i}^2 - 2(y_{i}\theta_{0} + y_{i}\theta_{1}X_{1i}) + y_{i}^2
$$

Next, we compute the derivative of the cost function with respect to $ \theta_0 $:

$$
\frac{\partial J}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^{m} (\theta_0 + \theta_1X_{1i} - y_i)
$$

For $ \theta_1 $, the derivative is:

$$
\frac{\partial J}{\partial \theta_1} = \frac{1}{m} \sum_{i=1}^{m} (\theta_0 + \theta_1X_{1i} - y_i)X_{1i}
$$

The general form for any weight $ \theta_j $ is:

$$
\frac{1}{m} \sum_{i=1}^{m} (f(x_{i}) - y_{i}) X_{ji}
$$


### 1.4. Vectorized Form

In practical implementations, especially when dealing with large datasets, it’s more efficient to use vectorized operations rather than loops. This allows the operations to be performed over entire matrices, leveraging the power of matrix multiplication for faster computations.

The vectorized form of the gradient is:

$$
\frac{1}{m} X^T \times (X \times \theta - Y)
$$

Where:
- $ X $ is the matrix of input features (size $ m \times k $, where $ m $ is the number of samples and $ k $ is the number of predictors),
- $ \theta $ is the vector of weights (size $k \times 1 $),
- $ Y $ is the vector of true values (size $ m \times 1 $).


### 1.5. Gradient Descent Algorithm - Updating

To implement gradient descent, we iteratively update the values of $ \theta $ by subtracting a fraction of the gradient at each step. This fraction is controlled by the learning rate $ \alpha $, which determines the step size.

The update rule for $ \theta_j $ is:

$$
\theta_j = \theta_j - \alpha \times \frac{1}{m} \sum_{i=1}^{m} (f(x_{i}) - y_{i}) X_{ji}
$$

Choosing the right learning rate is crucial for the success of gradient descent. If $ \alpha $ is too large, gradient descent might overshoot the minimum, while if it’s too small, the algorithm will take too long to converge. The goal is to find a balance that ensures steady progress toward the minimum without diverging.

# 2. Implementation
In this section, we will now compare the two approaches of implementing gradient descent: the **vectorized form** and the **iterative form**.

### 2.1. Vectorized Implementation
The **vectorized implementation** of gradient descent is highly efficient, as it leverages matrix operations to perform all the necessary calculations in one step. Using matrix multiplication, we can compute the predictions, errors, and parameter updates simultaneously for all training samples. This approach reduces the computational overhead by avoiding explicit loops over the dataset, making it much faster and scalable for larger datasets.

In the code below, the `gradientDescentVec` function performs gradient descent by computing the gradient using the entire dataset at once and updating the parameter vector `theta` accordingly.

In [2]:
def gradientDescentVec(X, y, alpha, iterations):
    import numpy as np
    
    # Add a column of ones to the input matrix X for the bias term (theta_0)
    app_X = np.ones((X.shape[0], 1))
    X = np.hstack((app_X, X))
    
    # Initialize the parameter vector theta with zeros
    theta = np.zeros((X.shape[1], 1))
    
    # Perform gradient descent for the specified number of iterations
    for i in range(iterations):
        # Update theta using the vectorized gradient descent formula
        theta = theta - alpha * (1/y.shape[0]) * (np.dot(X.T, (np.dot(X, theta) - y)))
    
    return theta

### 2.2. Iterative Implementation

The iterative implementation is more intuitive and closely follows the traditional step-by-step gradient descent procedure. In this version, we manually compute the predicted value, the error, and the parameter updates for each data point individually. This approach involves looping over each training sample and updating the parameters sequentially.

While easier to understand, this method can be much slower for large datasets, as it performs the calculations in a nested loop (over both samples and features). The `gradientDescent` function below implements this iterative approach.

In [3]:
def gradientDescent(X, y, alpha, iterations):
    import numpy as np
    
    # Add a column of ones to the input matrix X for the bias term (theta_0)
    app_X = np.ones((X.shape[0], 1))
    X = np.hstack((app_X, X))
    
    # Initialize the parameter vector theta with zeros
    theta = np.zeros((X.shape[1], 1))
    
    # Perform gradient descent iteratively
    for _ in range(iterations):
        theta_update = np.array([0]*X.shape[0]*X.shape[1]).reshape(X.shape[1], X.shape[0])
        
        # Loop through each data point
        for i in range(X.shape[0]):
            pred_vals = X[i, :]
            # Calculate the prediction for the current sample
            prediction = sum([temp_theta*temp_X for temp_theta, temp_X in zip(theta.T.flatten().tolist(), list(pred_vals))])
            pred_err = prediction - y[i]
            
            # Calculate the update for each parameter
            for j in range(X.shape[1]):
                temp_theta_update = pred_err * pred_vals[j]
                theta_update[j, i] = temp_theta_update
                
        # Update theta by averaging across all data points
        theta_update = np.sum(theta_update, axis=1)
        theta_update = theta_update / X.shape[0]
        theta = theta - alpha * theta_update.reshape(-1, 1)
        
    return theta

# 3.  Comparing the Two Approaches: Vectorized vs Iterative Gradient Descent

Now, let's compare the execution time of the vectorized and iterative implementations of gradient descent. We'll use the `%%time` magic command to measure the time it takes for each approach to run on a real dataset.

We will use the **Diabetes dataset** from `sklearn.datasets` for this comparison. This dataset contains 10 features and 442 samples, making it suitable for evaluating the performance difference between the two methods.


#### Timing the Iterative Implementation

Let's first measure the time taken by the **iterative** version of gradient descent.

In [6]:
%%time
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)

theta_iterative = gradientDescent(X, y.reshape(-1, 1), alpha=0.001, iterations=150)
theta_iterative

CPU times: total: 1.59 s
Wall time: 1.61 s


array([[21.13227376],
       [ 0.09649548],
       [ 0.02845023],
       [ 0.33065385],
       [ 0.25152262],
       [ 0.12353167],
       [ 0.09964932],
       [-0.20832127],
       [ 0.25519683],
       [ 0.31321041],
       [ 0.20956109]])

#### Timing the Vectorized Implementation
Now, we'll measure the time taken by the vectorized implementation.

In [7]:
%%time
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y = True)

theta_Vec = gradientDescentVec(X, y.reshape(-1, 1), .001, 150)
theta_Vec

CPU times: total: 15.6 ms
Wall time: 15.1 ms


array([[21.20080773],
       [ 0.10314334],
       [ 0.02359212],
       [ 0.32205638],
       [ 0.24243012],
       [ 0.11636862],
       [ 0.09550798],
       [-0.21677578],
       [ 0.23632567],
       [ 0.31073748],
       [ 0.21000291]])

### Conclusion
By comparing the output times from both methods, we can see how the vectorized approach performs significantly faster than the iterative one, especially as the dataset size increases. The vectorized approach leverages NumPy’s optimized operations for matrix multiplication, making it more suitable for large-scale data, while the iterative method may become slower as the number of samples and features grows.

It should also be noted that the produced weights (theta matrices) are nearly identical in both implementations. This indicates that both methods converge to the same solution, confirming that the gradient descent algorithm is functioning correctly in both cases. The key advantage of the vectorized approach lies in its computational efficiency, which becomes increasingly important with larger datasets and higher-dimensional feature spaces.

